In [11]:
import pandas as pd
import folium
from pathlib import Path
from datetime import datetime, timedelta

In [12]:
# ---------- 1. Load GTFS core files ----------
gtfs_path = Path("..") / "data" / "raw" /"budapest_gtfs"

In [13]:
# -------------------- Load GTFS core tables --------------------
stops = pd.read_csv(gtfs_path / "stops.txt")
trips = pd.read_csv(gtfs_path / "trips.txt")
stop_times = pd.read_csv(gtfs_path / "stop_times.txt")
routes = pd.read_csv(gtfs_path / "routes.txt")

In [14]:
# 2. Identify 3 busiest routes #
st_with_route = stop_times.merge(trips[['trip_id', 'route_id']], on='trip_id', how='left')
route_counts  = (st_with_route.groupby('route_id').size()
                 .sort_values(ascending=False))
top_route_ids = route_counts.head(3).index.tolist()

if not top_route_ids:
    raise ValueError("No route has stop_times in this GTFS feed!")

In [15]:
# 3.  Pick first route with at least one usable trip + stop_times

valid_trip_id  = None
primary_route_id = None

for rid in top_route_ids:
    for tid in trips[trips['route_id'] == rid]['trip_id']:
        if not stop_times[stop_times['trip_id'] == tid].empty:
            valid_trip_id  = tid
            primary_route_id = rid
            break
    if valid_trip_id:
        break

if valid_trip_id is None:
    raise ValueError("Couldn’t find any trip_id with stop_times in the top routes.")

In [16]:
# 4.  Prepare stop sequence for the chosen trip

trip_st = (stop_times[stop_times['trip_id'] == valid_trip_id]
           .sort_values('stop_sequence')
           .merge(stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id', how='left'))

def to_today(hms: str):
    try:
        h, m, s = map(int, hms.split(':'))
        h = h % 24                     # wrap 24–47→0–23
        base = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        return base + timedelta(hours=h, minutes=m, seconds=s)
    except:
        return None

trip_st['arrival_dt'] = trip_st['arrival_time'].astype(str).apply(to_today)
trip_st = trip_st.dropna(subset=['arrival_dt'])
if trip_st.empty:
    raise ValueError(f"No usable stop_times for trip {valid_trip_id}")

In [17]:
# 5.  Simulate current vehicle position (last stop already passed)

now = datetime.now()
past = trip_st[trip_st['arrival_dt'] <= now]

if past.empty:                     # vehicle hasn’t started yet → first stop
    veh_lat, veh_lon = trip_st.iloc[0][['stop_lat', 'stop_lon']]
    veh_stop         = trip_st.iloc[0]['stop_id']
else:                              # vehicle underway → last completed stop
    veh_lat, veh_lon = past.iloc[-1][['stop_lat', 'stop_lon']]
    veh_stop         = past.iloc[-1]['stop_id']

route_short = routes.loc[routes['route_id'] == primary_route_id,
                         'route_short_name'].iloc[0]

In [18]:
# 6.  Build polylines for the top-3 routes (first valid trip per route)

def route_coords(rid):
    for tid in trips[trips['route_id'] == rid]['trip_id']:
        seq = stop_times[stop_times['trip_id'] == tid]
        if not seq.empty:
            seq = (seq.sort_values('stop_sequence')
                      .merge(stops[['stop_id', 'stop_lat', 'stop_lon']],
                             on='stop_id'))
            return seq[['stop_lat', 'stop_lon']].values.tolist()
    return []

route_lines = {rid: route_coords(rid) for rid in top_route_ids}

In [19]:
# 7.  Create Folium map

center = [47.4979, 19.0402]  # Budapest city center
fmap   = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")
colors = ["red", "blue", "green"]

for idx, rid in enumerate(top_route_ids):
    coords = route_lines[rid]
    if coords:
        r_short = routes.loc[routes['route_id'] == rid,
                             'route_short_name'].iloc[0]
        folium.PolyLine(
            coords,
            color=colors[idx % len(colors)],
            weight=3, opacity=0.8,
            popup=f"Route {r_short}"
        ).add_to(fmap)

# vehicle marker
folium.Marker(
    location=[veh_lat, veh_lon],
    popup=(f"Simulated vehicle<br>"
           f"Route {route_short}<br>"
           f"Stop {veh_stop}<br>"
           f"{now.strftime('%H:%M')}"),
    icon=folium.Icon(color="purple", icon="train", prefix="fa")
).add_to(fmap)

In [24]:
# 8.  Save and display link


map_path = "../maps/budapest_transport_live_sim.html"
fmap.save(map_path)
map_path

'../maps/budapest_transport_live_sim.html'